In [1]:
import numpy as np
import pandas as pd

In [2]:
!pwd

/Users/chenghao/ResearchHub/repos/udao-spark-optimizer/playground/assets/data_stats


In [5]:
def regroup(original_bounds, n=50):
    """
    Transform an n-sized equi-height histogram into a 50-bin equi-height histogram by interpolating within each bin,
    assuming each bin has an equal number of data points.
    
    Args:
    original_bounds (list): The boundaries of the original histogram bins.
    N (int): The desired number of bins in the new histogram (50 by default).
    
    Returns:
    tuple: Tuple containing the new bin boundaries.
    """
    # Determine the number of original bins
    m = len(original_bounds) - 1
    rate = n / m
    new_bounds = [0] * (n + 1)
    new_bounds[0] = original_bounds[0]
    j = 0 # original bin index
    for i in range(1, n):
        while (j + 1) * rate < i:
            j += 1
        offset = i - j * rate
        # print(f"i: {i}, j: {j}, offset: {offset}")
        new_bounds[i] = original_bounds[j] + offset / rate * (original_bounds[j + 1] - original_bounds[j])
    new_bounds[n] = original_bounds[-1]
    return new_bounds


# Example usage
original_bounds = [1, 3, 4, 5, 6, 8]
new_bounds = regroup(original_bounds, n = 10)
print("New Bounds:", new_bounds)
new_bounds = regroup(original_bounds, n = 2)
print("New Bounds:", new_bounds)
new_bounds = regroup(original_bounds, n = 3)
print("New Bounds:", new_bounds)


New Bounds: [1, 2.0, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 7.0, 8]
New Bounds: [1, 4.5, 8]
New Bounds: [1, 3.666666666666667, 5.333333333333334, 8]


In [6]:
from datetime import datetime

def days_since_epoch(date_str):
    """
    Calculate the number of days from January 1, 1970, to the given date.
    
    Args:
    date_str (str): Date in "yyyy-mm-dd" format.
    
    Returns:
    int: Number of days between 1970-01-01 and the given date.
    """
    epoch_start = datetime(1970, 1, 1)
    given_date = datetime.strptime(date_str, '%Y-%m-%d')
    
    # Calculate the difference in days
    delta = given_date - epoch_start
    return delta.days

def regroup_to_50(dtype, bins_str, distinct_count):
    if distinct_count == 1 or bins_str == "[]":
        return [0.0] * 51
    if dtype == "date":
        bins = list(map(days_since_epoch, bins_str[1:-1].split(", ")))
    else:
        bins = list(map(float, bins_str[1:-1].split(", ")))
    
    if distinct_count < 51:
        ret_bins = bins 
        while len(ret_bins) < 51:
            ret_bins += bins
        ret_bins = sorted(ret_bins[:51])
        return ret_bins
        
    return regroup(bins, n=50)

,table,column,dtype,bins,distinct_count,hists
0,customer,c_custkey,bigint,"[1.0, 58629.0, 117724.0, 176935.0, 235656.0, 2...",13828764,"[1.0, 299788.16, 599683.48, 899274.1599999999,..."
1,customer,c_name,string,[],15000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,customer,c_address,string,[],15000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,customer,c_nationkey,bigint,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...",25,"[0.0, 0.0, 0.0, 1.0, 1.0, 2.0, 2.0, 3.0, 3.0, ..."
4,customer,c_phone,string,[],15000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


For TPCH

In [3]:
# Load data
bm = "tpch"
df = pd.read_csv(f"raw_{bm}_hist.csv")

# Regroup data
df["hists"] = df[["dtype", "bins", "distinct_count"]].apply(lambda x: regroup_to_50(x["dtype"], x["bins"], x["distinct_count"]), axis=1)
df.to_csv(f"regrouped_{bm}_hist.csv", index=False)
df.head()

,table,column,dtype,bins,distinct_count
0,customer,c_custkey,bigint,"[1.0, 58629.0, 117724.0, 176935.0, 235656.0, 2...",13828764
1,customer,c_name,string,[],15000000
2,customer,c_address,string,[],15000000
3,customer,c_nationkey,bigint,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...",25
4,customer,c_phone,string,[],15000000


For TPCDS

In [84]:
# Load data
bm = "tpcds"
df = pd.read_csv(f"raw_{bm}_hist.csv")
# Regroup data
df["hists"] = df[["dtype", "bins", "distinct_count"]].apply(lambda x: regroup_to_50(x["dtype"], x["bins"], x["distinct_count"]), axis=1)
df.to_csv(f"regrouped_{bm}_hist.csv", index=False)


In [77]:
df[df.column == "d_date"]["bins"].values[0]

'[1900-01-02, 1900-10-17, 1901-08-01, 1902-05-07, 1903-02-19, 1903-12-04, 1904-09-17, 1905-07-02, 1906-04-16, 1907-01-29, 1907-11-13, 1908-08-27, 1909-06-11, 1910-03-26, 1911-01-08, 1911-10-23, 1912-08-06, 1913-05-21, 1914-03-01, 1914-12-14, 1915-09-28, 1916-07-12, 1917-04-26, 1918-02-08, 1918-11-23, 1919-09-07, 1920-06-21, 1921-04-05, 1922-01-18, 1922-11-02, 1923-08-17, 1924-05-31, 1925-03-15, 1925-12-20, 1926-10-04, 1927-07-18, 1928-05-07, 1929-02-18, 1929-12-02, 1930-09-15, 1931-06-29, 1932-04-11, 1933-01-23, 1933-11-06, 1934-08-20, 1935-06-03, 1936-03-16, 1936-12-28, 1937-10-18, 1938-08-01, 1939-05-15, 1940-02-26, 1940-12-09, 1941-09-23, 1942-07-08, 1943-04-22, 1944-02-04, 1944-11-12, 1945-08-27, 1946-06-11, 1947-03-26, 1948-01-08, 1948-10-22, 1949-08-06, 1950-05-21, 1951-03-05, 1951-12-18, 1952-10-01, 1953-07-16, 1954-04-30, 1955-02-10, 1955-11-27, 1956-09-12, 1957-06-19, 1958-04-05, 1959-01-20, 1959-11-06, 1960-08-12, 1961-05-29, 1962-03-15, 1962-12-30, 1963-10-16, 1964-07-22, 19

For JOB

In [7]:
# Load data
bm = "job"
df = pd.read_csv(f"raw_{bm}_hist.csv")
# Regroup data
df["hists"] = df[["dtype", "bins", "distinct_count"]].apply(lambda x: regroup_to_50(x["dtype"], x["bins"], x["distinct_count"]), axis=1)
df.to_csv(f"regrouped_{bm}_hist.csv", index=False)

In [12]:
df[df.dtype != "string"]

,table,column,dtype,bins,distinct_count,hists
0,aka_name,id,int,"[1.0, 3511.0, 7132.0, 10626.0, 14210.0, 17682....",861761,"[1.0, 17971.68, 36049.32, 54071.439999999995, ..."
1,aka_name,person_id,int,"[4.0, 16885.0, 33218.0, 49812.0, 66344.0, 8352...",621158,"[4.0, 84817.95999999999, 165829.19999999998, 2..."
8,aka_title,id,int,"[1.0, 1571.0, 3218.0, 4866.0, 6427.0, 7942.0, ...",361472,"[1.0, 8068.96, 16293.84, 24230.96, 31832.52, 3..."
9,aka_title,movie_id,int,"[0.0, 97178.0, 186882.0, 303896.0, 408440.0, 5...",193791,"[0.0, 535126.4, 999935.76, 1527547.0, 1653508...."
12,aka_title,kind_id,int,"[1.0, 2.0, 3.0, 4.0, 6.0, 7.0]",6,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
13,aka_title,production_year,int,"[1888.0, 1900.0, 1903.0, 1907.0, 1909.0, 1910....",128,"[1888.0, 1903.16, 1909.08, 1911.12, 1913.16, 1..."
15,aka_title,episode_of_id,int,"[10.0, 187.0, 432.0, 465.0, 550.0, 776.0, 1002...",747,"[10.0, 514.3000000000001, 1106.16, 1479.06, 20..."
16,aka_title,season_nr,int,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...",41,"[1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 3.0, 3.0, 3.0, ..."
17,aka_title,episode_nr,int,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...",165,"[1.0, 2.18, 3.3600000000000003, 4.54, 5.720000..."
20,cast_info,id,int,"[1.0, 143253.0, 284549.0, 425258.0, 568988.0, ...",36244344,"[1.0, 727645.76, 1448967.96, 2172776.4, 289901..."
